## Homework 08: Classification

**Due:** Midnight on July 6th, BUT no late points will be charged if you get it in by the last late deadline. 

### Overview

In this final homework before starting our course project, we will introduce the essential machine learning paradigm of **classification**. We will work with the **UCI Adult** dataset. This is a binary classification task.

As we’ve discussed in this week’s lessons, the classification workflow is similar to what we’ve done for regression, with a few key differences:
- We use `StratifiedKFold` instead of plain `KFold` so that every fold keeps the original class proportions.
- We use classification metrics (e.g., accuracy, precision, recall, F1-score for binary classification) instead of regression metrics.
- We could explore misclassified instances through a confusion matrix (though we will not do that in this homework).

For this assignment, you’ll build a gradient boosting classification using `HistGradientBoostingClassifier` (HGBC) and explore ways of tuning the hyperparameters, including using the technique of early stopping, which basically avoiding have to tune the number of estimators (called `max_iter` in HGBC). 

HGBC has many advantages, which we explain below. 


### Grading

There are 7 graded problems, each worth 7 points, and you get 1 point free if you complete the assignment. 

In [4]:
!pip install tqdm
!pip install optuna
# General utilities
import os
import io
import time
import zipfile
import requests
from collections import Counter

# Data handling and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import display
 
# Data source
from sklearn.datasets import fetch_openml

 
# scikit-learn core tools 
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    StratifiedKFold,
    RandomizedSearchCV
)

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder

 
# Import model 
from sklearn.ensemble import HistGradientBoostingClassifier
 
# Metrics
from sklearn.metrics import balanced_accuracy_score, classification_report
 
# Distributions for random search
from scipy.stats import loguniform, randint, uniform

# pandas dtypes helpers
from pandas.api.types import is_numeric_dtype, is_categorical_dtype
from pandas import CategoricalDtype

# Optuna Hyperparameter Search tool    (may need to be installed)
import optuna


# Misc

random_seed = 42

def format_hms(seconds):
    return time.strftime("%H:%M:%S", time.gmtime(seconds))



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [optuna]2m5/6 [optuna]]my]


/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Prelude 1: Load and Preprocess the UCI Adult Income Dataset

- Load the dataset from sklearn
- Preliminary EDA
- Feature Engineering 

In [5]:
# Load and clean
df = fetch_openml(name='adult', version=2, as_frame=True).frame

df.replace("?", np.nan, inplace=True)            # Some datasets use ? instead of Nan for missing data

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   age             48842 non-null  int64   
 1   workclass       46043 non-null  category
 2   fnlwgt          48842 non-null  int64   
 3   education       48842 non-null  category
 4   education-num   48842 non-null  int64   
 5   marital-status  48842 non-null  category
 6   occupation      46033 non-null  category
 7   relationship    48842 non-null  category
 8   race            48842 non-null  category
 9   sex             48842 non-null  category
 10  capital-gain    48842 non-null  int64   
 11  capital-loss    48842 non-null  int64   
 12  hours-per-week  48842 non-null  int64   
 13  native-country  47985 non-null  category
 14  class           48842 non-null  category
dtypes: category(9), int64(6)
memory usage: 2.7 MB


#### Check: Is the dataset imbalanced?

In [6]:
print(df['class'].value_counts(normalize=True))

class
<=50K    0.760718
>50K     0.239282
Name: proportion, dtype: float64


**YES:** It looks like this dataset is somewhat imbalanced. Therefore, we will 
1. Tell the model to compensate during training by setting `class_weight='balanced'` when defining the model;
2. Evaluate it `balanced_accuracy` instead of `accuracy` and with class-aware metrics (precision, recall, F1); and
3. [Optional] Adjust the probability threshold instead of relying on raw accuracy alone after examining the precision-recall trade-off you observe at 0.5.
    

### Feature Engineering

Based on the considerations in **Appendix One**, we'll make the following changes to the dataset to facilitate training:


1. Drop `fnlwgt` and `education`.   
3. Replace `capital-gain` and `capital-loss` by their difference `capital_net` and add a log-scaled version `capital_net_log`.


In [7]:
# Drop the survey-weight column
df_eng = df.drop(columns=["fnlwgt"])

# Keep only the ordinal education feature
df_eng = df_eng.drop(columns=["education"])      # retain 'education-num'

# Combine capital gains and losses, add a log-scaled variant
df_eng["capital_net"]     = df_eng["capital-gain"] - df_eng["capital-loss"]
df_eng["capital_net_log"] = np.log1p(df_eng["capital_net"].clip(lower=0))
df_eng = df_eng.drop(columns=["capital-gain", "capital-loss"])

# check
df_eng.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   age              48842 non-null  int64   
 1   workclass        46043 non-null  category
 2   education-num    48842 non-null  int64   
 3   marital-status   48842 non-null  category
 4   occupation       46033 non-null  category
 5   relationship     48842 non-null  category
 6   race             48842 non-null  category
 7   sex              48842 non-null  category
 8   hours-per-week   48842 non-null  int64   
 9   native-country   47985 non-null  category
 10  class            48842 non-null  category
 11  capital_net      48842 non-null  int64   
 12  capital_net_log  48842 non-null  float64 
dtypes: category(8), float64(1), int64(4)
memory usage: 2.2 MB


#### Separate target and split

Create the feature set `X` and the target set `y` (using `class` as the target) and split the dataset into 80% training and 20% testing sets, making sure to stratify.

In [8]:

X = df_eng.drop(columns=["class"])
y = (df_eng["class"] == ">50K").astype(int)

# Split (with stratification)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=random_seed,
    stratify=y                           # So same proportion of classes in train and test sets
)

print("Train:", X_train.shape, y_train.shape)
print("Test :", X_test.shape,  y_test.shape)

Train: (39073, 12) (39073,)
Test : (9769, 12) (9769,)


### Prelude 2: Create a data pipeline and the `HistGradientBoostingClassifier` model

Histogram-based gradient boosting improves on the standard version by:

* **Histogram splits:** bins each feature into ≤ `max_bins` quantiles (i.e., each bin is approximately the same size) and tests splits only between bins, slashing compute time and scaling to large data sets. Default for `max_bins` = 255. 
* **Native NaN handling:** treats missing values as their own bin—no imputation needed.
* **Native Categorical Support**: accepts integer-encoded categories directly and tests “category c vs. all others” splits, eliminating one-hot blow-ups and fake orderings.
* **Built-in early stopping:** stops training after no improvement in validation loss after `n_iter_no_change` rounds. `tol` defines "improvement" (default is 1e-7). 
* **Leaf shrinkage:** adds `l2_regularization`, which ridge-shrinks each leaf value (without changing tree shape) so tiny, noisy leaves have less effect.

>**Summary:**  Histogram-based GB trades a tiny approximation error (binning) for a **huge speed-up** and adds extra conveniences, making it the preferred choice for large tabular data sets. Tuning workflow relies on **Early stopping** to stop training before overfitting occurs. 

In [9]:
# Define a baseline model 

HGBC_model = HistGradientBoostingClassifier(
    # tree structure and learning rate
    learning_rate=0.1,            # These 5 parameters are at defaults for our baseline training in Problem 1             
    max_leaf_nodes=31,            # but will be tuned by randomized search in Problem 2 and Optuna in Problem 3               
    max_depth=None,               
    min_samples_leaf=20,          
    l2_regularization=0.0,        

    # bins and iteration
    max_bins=255,                 # default
    max_iter=500,                 # high enough for early stopping
    early_stopping=True,
    n_iter_no_change=20,
    validation_fraction=0.2,      # 20% monitored for early stopping
    tol=1e-7,                     # default tolerance for validation improvement

    # class imbalance
    class_weight="balanced",

    random_state=random_seed,
    verbose=0
)


### Create a pipeline appropriate for HGBC 

**Why use a `Pipeline` instead of encoding in the dataset first?**

* **Avoid data leakage.** In each CV fold, the `OrdinalEncoder` is refit only on that fold’s training data, so the validation split never influences the encoder.
* **Single, reusable object.** The pipeline bundles preprocessing + model, letting you call `fit`/`predict` on raw data anywhere (CV, Optuna, production) with identical behavior.
* **Compatible with search tools.** `cross_validate`, `GridSearchCV`, and Optuna expect an estimator that can be cloned and refit; a pipeline meets that requirement automatically.

Put simply, the pipeline gives you leak-free evaluation and portable, hassle-free tuning without extra code.


In [10]:
enc = OrdinalEncoder(
    handle_unknown="use_encoded_value",   # Allow unseen categories during transform
    unknown_value=-1,                     # Code for unseen categories
    encoded_missing_value=-2,             # Code for missing values (NaN)
    dtype=np.int64                        # Needed for HistGradientBoostingClassifier
)

# Categorical features
cat_cols = X.select_dtypes(exclude=["number"]).columns.tolist()

# Numeric features (everything that isn’t object / category)
num_cols = X.select_dtypes(include=["number"]).columns.tolist()

preprocess = ColumnTransformer(
    [("cat", enc, cat_cols),
     ("num", "passthrough", num_cols)]
)

pipelined_model = Pipeline([
    ("prep", preprocess),
    ("gb",   HGBC_model)
])

## Problem 1: Baseline Cross-Validation with F1

In this problem, you will run a baseline cross-validation evaluation of your `HistGradientBoostingClassifier` pipeline, using `HGBC_model` defined above. 

**Background:**

* Since the Adult dataset is imbalanced (about 24% positives, 76% negatives), accuracy alone is not reliable.
* We will use the **F1 score** as the evaluation metric, since it balances precision (avoiding false positives) and recall (avoiding false negatives) in a single measure. This is a fairer metric for imbalanced classification, where both types of error matter.
* We will apply **5-fold stratified cross-validation** to make sure each fold has the same proportion of the classes as the original dataset.
* Repeated cross-validation is optional and not required here, because the Adult dataset is large and `HistGradientBoostingClassifier` is robust to small sampling differences. 

**Instructions:**

1. Set up a `StratifiedKFold` cross-validation object with 5 splits, shuffling enabled, and `random_state=random_seed`.
2. Use `cross_val_score` to estimate the mean F1 score and its standard deviation across the folds.
3. Print out the mean and standard deviation of the F1 score, rounded to 4 decimal places.
4. Answer the graded question.


In [ ]:
# Your code here

from sklearn.model_selection import StratifiedKFold, cross_val_score

#Set up 5-fold stratified CV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)

#Estimate F1 scores
f1_scores = cross_val_score(
    estimator=HGBC_model,
    X=X,
    y=y,
    scoring='f1',
    cv=cv
)

#Print mean & std 
print(f"Mean F1 score: {f1_scores.mean():.4f}")
print(f"Std  of F1 score: {f1_scores.std():.4f}")



Mean F1 score: 0.7130
Std  of F1 score: 0.0056


### Problem 1 Graded Answer

Set `a1` to the mean F1 score of the baseline model. 

In [12]:
 # Your answer here

a1 = round(f1_scores.mean(), 4)

In [13]:
# DO NOT change this cell in any way

print(f'a1 = {a1:.4f}')

a1 = 0.7130


## Problem 2: Hyperparameter Optimization with Randomized Search for F1

In this problem, you will tune your `pipelined_model` using `RandomizedSearchCV` to identify the best combination of tree structure and learning rate parameters that maximize the **F1 score**.

**Background:**
The F1 score is our main metric because it balances precision and recall on an imbalanced dataset. Optimizing hyperparameters for F1 ensures we manage both false positives and false negatives in a single measure.

**Instructions:**

1. Set up a randomized search over the following hyperparameter ranges, using appropriate random-number distributions:

   * `learning_rate` (log-uniform between 1e-3 and 0.3)
   * `max_leaf_nodes` (integer from 16 to 256)
   * `max_depth` (integer from 2 to 10)
   * `min_samples_leaf` (integer from 10 to 200)
   * `l2_regularization` (uniform between 0.0 and 2.0)
2. Use **5-fold stratified cross-validation**, with the same settings as in Problem 1.
3. Set `n_iter` to at least 100 trials. More trials will generally yield better results, if your time and machine allow.
4. After running the search, show a neatly formatted table of the top 5 results, using `display(...)` showing their mean F1 scores, standard deviation, and the chosen hyperparameter values.
5. Answer the graded question.




In [15]:
# Your code here

from scipy.stats import loguniform, randint, uniform
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
import pandas as pd

# 1. Corrected hyperparameter distributions (use 'gb__' prefix)
param_dist = {
    'gb__learning_rate':     loguniform(1e-3, 0.3),
    'gb__max_leaf_nodes':     randint(16, 257),
    'gb__max_depth':          randint(2, 11),
    'gb__min_samples_leaf':   randint(10, 201),
    'gb__l2_regularization':  uniform(0.0, 2.0)
}

# 2. Same CV as before
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)

# 3. Set up RandomizedSearchCV
rs = RandomizedSearchCV(
    estimator=pipelined_model,         # your pipeline
    param_distributions=param_dist,
    n_iter=100,
    scoring='f1',
    cv=cv,
    random_state=random_seed,
    n_jobs=-1,
    verbose=1
)

# 4. Fit
rs.fit(X, y)

# 5. Display top 5 configurations
results = pd.DataFrame(rs.cv_results_)
cols = ['mean_test_score', 'std_test_score'] + [c for c in results.columns if c.startswith('param_')]
top5 = results[cols].sort_values('mean_test_score', ascending=False).head(5)
display(top5)

# 6. Assign the best mean F1 to a2
a2 = round(rs.best_score_, 4)
print(f"Best mean F1 (a2): {a2}")



Fitting 5 folds for each of 100 candidates, totalling 500 fits


,mean_test_score,std_test_score,param_gb__l2_regularization,param_gb__learning_rate,param_gb__max_depth,param_gb__max_leaf_nodes,param_gb__min_samples_leaf
59,0.713984,0.004860,1.495438,0.229549,2,61,190
55,0.713927,0.004683,0.973484,0.175597,3,36,39
19,0.713154,0.004955,1.829919,0.127541,3,48,57
99,0.712859,0.005075,1.900124,0.226344,4,126,153
98,0.712467,0.003557,0.021675,0.174880,10,214,37


Best mean F1 (a2): 0.714


### Problem 2 Graded Answer

Set `a2` to the mean F1 score of the best model found. 

In [16]:
 # Your answer here

a2 = round(rs.best_score_, 4)                 # replace 0 with your answer, may copy from the displayed results

In [17]:
# DO NOT change this cell in any way

print(f'a2 = {a2:.4f}')

a2 = 0.7140


## Problem 3: Hyperparameter Optimization with Optuna for F1

In this problem, you will explore **Optuna**, a powerful hyperparameter optimization framework, to identify the best combination of hyperparameters that maximize the F1 score of your `pipelined_model`.

**Background:**
Optuna uses a smarter sampling strategy than grid search or randomized search, allowing you to explore the hyperparameter space more efficiently. It also supports *pruning*, which can stop unpromising trials early to save time. This makes it a popular SOTA optimization tool.

**Before you start** browse the [Optuna documentation](https://optuna.org) and view the [tutorial video](https://optuna.readthedocs.io/en/stable/tutorial/index.html). 

As before, we focus on the **F1 score** because it balances precision and recall, making it more robust on an imbalanced dataset.

**Instructions:**

1. Define an Optuna objective function to optimize F1 score, sampling the exact same hyperparameter ranges you did in Problem 2 and using the same CV settings.  
3. Set up an Optuna study with a reasonable number of trials (e.g., 100–500 depending on runtime resources--on my machine Optuna runs about 10x faster than randomized search for the same number of trials, but YMMV).
4. After running the optimization, `display` a clean table with the top 5 trials showing their F1 scores and corresponding hyperparameter settings.
5. Answer the graded question. 

**Note:**  There are many resources on Optuna you can find on the web, but for this problem, you have my permission to let ChatGPT write the code for you. 

In [22]:
import optuna
from sklearn.model_selection import StratifiedKFold, cross_val_score
import pandas as pd

# 1. CV object
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)

# 2. Objective
def objective(trial):
    params = {
        'gb__learning_rate':     trial.suggest_loguniform('learning_rate', 1e-3, 0.3),
        'gb__max_leaf_nodes':    trial.suggest_int('max_leaf_nodes', 16, 256),
        'gb__max_depth':         trial.suggest_int('max_depth', 2, 10),
        'gb__min_samples_leaf':  trial.suggest_int('min_samples_leaf', 10, 200),
        'gb__l2_regularization': trial.suggest_uniform('l2_regularization', 0.0, 2.0)
    }
    pipelined_model.set_params(**params)
    scores = cross_val_score(pipelined_model, X, y, scoring='f1', cv=cv, n_jobs=-1)
    return scores.mean()

# 3. Run study
study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=random_seed),
    pruner=optuna.pruners.MedianPruner()
)
study.optimize(objective, n_trials=100, n_jobs=1, show_progress_bar=True)

# 4. Manually build a DataFrame of trials
records = []
for t in study.trials:
    records.append({
        'trial_number':      t.number,
        'mean_f1':           t.value,
        'learning_rate':     t.params['learning_rate'],
        'max_leaf_nodes':    t.params['max_leaf_nodes'],
        'max_depth':         t.params['max_depth'],
        'min_samples_leaf':  t.params['min_samples_leaf'],
        'l2_regularization': t.params['l2_regularization']
    })
trials_df = pd.DataFrame(records)

# Display the top 5 trials
top5 = trials_df.sort_values('mean_f1', ascending=False).head(5)
display(top5)

# 5. Assign the graded answer
a3 = round(study.best_value, 4)
print(f"Best F1 from Optuna (a3): {a3}")


[I 2025-07-08 03:41:14,704] A new study created in memory with name: no-name-7eaef0f4-3633-46b2-8e48-0a817eb6ceaa
  0%|          | 0/100 [00:00<?, ?it/s]/tmp/ipykernel_7726/1741534080.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gb__learning_rate':     trial.suggest_loguniform('learning_rate', 1e-3, 0.3),
/tmp/ipykernel_7726/1741534080.py:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'gb__l2_regularization': trial.suggest_uniform('l2_regularization', 0.0, 2.0)
Best trial: 0. Best value: 0.69529:   1%|          | 1/100 [00:17<29:27, 17.85s/it]/tmp/ipykernel_7726/1741534080.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in 

[I 2025-07-08 03:41:32,556] Trial 0 finished with value: 0.6952899580869166 and parameters: {'learning_rate': 0.008468008575248327, 'max_leaf_nodes': 245, 'max_depth': 8, 'min_samples_leaf': 124, 'l2_regularization': 0.31203728088487304}. Best is trial 0 with value: 0.6952899580869166.


Best trial: 0. Best value: 0.69529:   2%|▏         | 2/100 [00:32<25:47, 15.79s/it]/tmp/ipykernel_7726/1741534080.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gb__learning_rate':     trial.suggest_loguniform('learning_rate', 1e-3, 0.3),
/tmp/ipykernel_7726/1741534080.py:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'gb__l2_regularization': trial.suggest_uniform('l2_regularization', 0.0, 2.0)


[I 2025-07-08 03:41:46,911] Trial 1 finished with value: 0.6746784536793463 and parameters: {'learning_rate': 0.0024345423962016913, 'max_leaf_nodes': 29, 'max_depth': 9, 'min_samples_leaf': 124, 'l2_regularization': 1.416145155592091}. Best is trial 0 with value: 0.6952899580869166.


Best trial: 0. Best value: 0.69529:   3%|▎         | 3/100 [00:56<31:34, 19.54s/it]/tmp/ipykernel_7726/1741534080.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gb__learning_rate':     trial.suggest_loguniform('learning_rate', 1e-3, 0.3),
/tmp/ipykernel_7726/1741534080.py:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'gb__l2_regularization': trial.suggest_uniform('l2_regularization', 0.0, 2.0)


...

[I 2025-07-08 03:51:32,885] Trial 98 finished with value: 0.71397709838568 and parameters: {'learning_rate': 0.1334492978124086, 'max_leaf_nodes': 200, 'max_depth': 2, 'min_samples_leaf': 22, 'l2_regularization': 1.7562493590504484}. Best is trial 83 with value: 0.7150961500692044.


Best trial: 83. Best value: 0.715096: 100%|██████████| 100/100 [10:24<00:00,  6.25s/it]

[I 2025-07-08 03:51:39,571] Trial 99 finished with value: 0.7128454243442167 and parameters: {'learning_rate': 0.13974049803506472, 'max_leaf_nodes': 195, 'max_depth': 2, 'min_samples_leaf': 22, 'l2_regularization': 1.6165028214551105}. Best is trial 83 with value: 0.7150961500692044.


,trial_number,mean_f1,learning_rate,max_leaf_nodes,max_depth,min_samples_leaf,l2_regularization
83,83,0.715096,0.139972,160,4,16,0.313233
30,30,0.714618,0.225785,172,3,38,1.539161
31,31,0.714441,0.289079,172,3,33,1.532276
88,88,0.714276,0.232651,218,3,10,1.876012
73,73,0.714182,0.211761,215,3,12,0.128171


Best F1 from Optuna (a3): 0.7151


### Problem 3 Graded Answer

Set `a3` to the mean F1 score of the best model found. 

In [23]:
 # Your answer here

a3 = round(study.best_value, 4)                # replace 0 with your answer, may copy from the displayed results

In [24]:
# DO NOT change this cell in any way

print(f'a3 = {a3:.4f}')

a3 = 0.7151


## Problem 4: Final Model Evaluation on Test Set

In this problem, you will take the best hyperparameter configuration you found in your earlier experiments (Randomized Search or Optuna) and fully evaluate the resulting model on the test set.

**Background:**
When performing hyperparameter tuning, we typically optimize for a single metric (e.g., F1). However, before deployment, it is essential to check **all relevant metrics** on the final test set to understand the model’s behavior in a balanced way.

**Instructions:**

1. Take the best hyperparameters you found in Problems 2 or 3 and apply them to your `pipelined_model`.
2. Re-train this final tuned model on the **entire training set** (not just the folds).
3. Evaluate the final model on the heldout **test set**, reporting the following metrics:

   * Precision
   * Recall
   * F1 score
   * Balanced accuracy
4. Use `classification_report` **on the test set** to print precision, recall, and F1 score, and use `balanced_accuracy_score` separately to calculate and print balanced accuracy.
5. Answer the graded questions.

**Note:** We evaluate the metrics on the test set because it was never seen during training or hyperparameter tuning. This gives us an unbiased estimate of how the model will perform on truly unseen data. Evaluating on the training set would be misleading, because the model has already learned from that data and could appear artificially good.


In [29]:
# Your code here

from sklearn.metrics import classification_report, balanced_accuracy_score, precision_score, recall_score, f1_score

# 1. Retrieve your best hyperparameters (from Optuna study or RandomizedSearchCV)
#    If using Optuna:
best_params = study.best_trial.params
#    If instead you want to use RandomizedSearchCV, you could do:
# best_params = {k.replace('gb__', ''): v for k, v in rs.best_params_.items()}

# 2. Prefix them to match your pipeline step name
prefixed_params = {f'gb__{k}': v for k, v in best_params.items()}

# 3. Apply them to your pipeline
pipelined_model.set_params(**prefixed_params)

# 4. Retrain on the full training set
#    (assuming you have X_train, X_test, y_train, y_test already defined)
pipelined_model.fit(X_train, y_train)

# 5. Evaluate on the test set
y_pred = pipelined_model.predict(X_test)

# 6. Compute all metrics
precision    = precision_score(y_test, y_pred)
recall       = recall_score(y_test, y_pred)
f1           = f1_score(y_test, y_pred)
balanced_acc = balanced_accuracy_score(y_test, y_pred)

# 7. Print a classification report plus balanced accuracy
print("Classification Report:\n", classification_report(y_test, y_pred))
print(f"Balanced Accuracy: {balanced_acc:.4f}")

# 8. Assign your graded-question variables
a4_precision         = round(precision, 4)
a4_recall            = round(recall, 4)
a4_f1                = round(f1, 4)
a4_balanced_accuracy = round(balanced_acc, 4)

# 3. Get full classification report as dict
report = classification_report(y_test, y_pred, output_dict=True)

# Example prints of the rounded values:
print(f"a4_precision: {a4_precision}")
print(f"a4_recall: {a4_recall}")
print(f"a4_f1: {a4_f1}")
print(f"a4_balanced_accuracy: {a4_balanced_accuracy}")



Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.82      0.88      7431
           1       0.61      0.87      0.72      2338

    accuracy                           0.84      9769
   macro avg       0.78      0.85      0.80      9769
weighted avg       0.87      0.84      0.84      9769

Balanced Accuracy: 0.8470
a4_precision: 0.6092
a4_recall: 0.8695
a4_f1: 0.7165
a4_balanced_accuracy: 0.847


### Problem 4 Graded Questions

- Set `a4a` to the balanced accuracy score of the best model.
- Set `a4b` to the macro average precision of this model.
- Set `a4c` to the macro average recall score of the this model.

**Note:** Macro average takes the mean of each class’s precision/recall without considering how many samples each class has, which is appropriate for a balanced evaluation.

In [30]:
 # Your answer here

a4a = balanced_accuracy_score(y_test, y_pred)                # replace 0 with your answer, use variable or expression from above

In [27]:
# DO NOT change this cell in any way

print(f'a4a = {a4a:.4f}')

a4a = 0.8470


In [31]:
 # Your answer here

a4b = report['macro avg']['precision']                     # replace 0 with your answer, may copy from the displayed results

In [32]:
# DO NOT change this cell in any way

print(f'a4b = {a4b:.4f}')

a4b = 0.7809


In [33]:
 # Your answer here

a4c = report['macro avg']['recall']                    # replace 0 with your answer, may copy from the displayed results

In [34]:
# DO NOT change this cell in any way

print(f'a4c = {a4c:.4f}')

a4c = 0.8470


## Problem 5: Understanding Precision, Recall, F1, and Balanced Accuracy

**Tutorial**

In binary classification, you will often evaluate these key metrics:

* **Precision**: *Of all the positive predictions the model made, how many were actually correct?*

  * High precision = few false positives
  * Low precision = many false positives

* **Recall**: *Of all the actual positive cases, how many did the model correctly identify?*

  * High recall = few false negatives
  * Low recall = many false negatives

* **F1 score**: The harmonic mean of precision and recall, which balances them in a single measure.

  * F1 is **highest** when precision and recall are both high and similar in value.
  * If precision and recall are unbalanced, F1 will drop to reflect that imbalance.

* **Balanced accuracy**: The average of recall across both classes (positive and negative).

  * It ensures the classifier is performing reasonably well on *both* groups, correcting for class imbalance.
  * Balanced accuracy is especially important if the classes are very unequal in size.

**Typical trade-offs to remember:**

* **Higher recall, lower precision**: the model finds most true positives but also mislabels some negatives as positives
* **Higher precision, lower recall**: the model is strict about positive predictions, but misses some true positives
* **Balanced precision and recall (good F1)**: a practical compromise
* **Balanced accuracy**: checks fairness across both classes

###  Problem 5 Graded Question (multiple choice)

A bank uses your model to identify customers earning over $50K for a premium product invitation. Based on your final test set evaluation, including macro-averaged precision and recall, which of the following best describes what might happen?

(1) The bank will miss some eligible high-income customers, but will avoid marketing mistakes by sending invitations only to those it is  confident about.

(2) The bank will successfully reach most high-income customers, but will also waste resources sending invitations to some low-income customers.

(3) The bank will perfectly identify all high-income and low-income customers, resulting in no wasted invitations and no missed opportunities.


In [35]:
 # Your answer here

a5 = 1                     # replace 0 with one of 1, 2, or 3

In [36]:
# DO NOT change this cell in any way

print(f'a5 = {a5}')

a5 = 1


### Appendix One: Feature Engineering

Here are some practical feature-engineering tweaks worth considering (beyond simply ordinal-encoding the categoricals)

| Feature(s)                                                           | Why the tweak can help                                                                                                                                                     | How to do it (quick version)                                                                                                                                                    | Keep / drop?      |
| -------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ----------------- |
| **`fnlwgt`**                                                         | Survey sampling weight, not a predictor. Leaving it in often lets the model “cheat.”                                                                                       | `df = df.drop(columns=["fnlwgt"])`                                                                                                                                              | **Drop**          |
| **`education` *vs.* `education-num`**                                | They encode the **same** information twice (categorical label and its ordinal rank). Keeping both is redundant and can cause leakage of a perfectly predictive feature.    | Usually keep **only one**. For tree models `education-num` is simplest: `df = df.drop(columns=["education"])`                                                                   | **Drop one**      |
| **`capital-gain`, `capital-loss`**                                   | Highly skewed; most values are zero with a long upper tail. The sign (gain vs. loss) matters, but treating them separately wastes a feature slot.                          | 1) Combine: `df["capital_net"] = df["capital-gain"] - df["capital-loss"]`; 2) Log-transform to reduce skew: `df["capital_net_log"] = np.log1p(df["capital_net"].clip(lower=0))` | Replace originals |
| **`age`, `hours-per-week`**                                          | Continuous but with natural plateaus—trees handle splits fine, yet log or square-root scaling can soften extreme values; bucketing makes partial-dependence plots clearer. | Simple bucket: `df["age_bin"] = pd.cut(df["age"], bins=[16,25,35,45,55,65,90])` (optional)                                                                                      | Optional          |
| **Missing categories** (`workclass`, `occupation`, `native-country`) | HGB handles `-1`/`-2` codes fine, but you may want *explicit* “Missing” bucket for interpretability.                                                                       | Use `encoded_missing_value=-2` during encoding.                                                                                                            | Keep as is        |
| **Rare categories in `native-country`**                              | Hundreds of low-frequency countries dilute signal; grouping boosts stability.                                                                                              | Map infrequent categories to “Other”:                                                                                                                                           |                   |


#### Minimum set of tweaks (good baseline, low effort)

1. **Drop `fnlwgt`.**  
2. **Keep `education-num`, drop `education`.**  
3. **Combine `capital-gain` and `capital-loss` into `capital_net`** (optionally add a log-scaled version).  
4. Leave other numeric/categorical features as is; your histogram-GBDT will cope.


